In [1]:
import requests
from http import HTTPStatus
import dashscope
import os

# 设置 DashScope API Key
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

# 高德天气 API 的 天气工具定义（JSON 格式）
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name, e.g. 北京",
                },
                "adcode": {
                    "type": "string",
                    "description": "The city code, e.g. 110000 (北京)",
                }
            },
            "required": ["location"],
        },
    },
}

def get_current_weather(location: str, adcode: str = None):
    """调用高德地图API查询天气"""
    gaode_api_key = os.getenv("AMAP_MAPS_API_KEY")  # 替换成你的高德API Key
    base_url = "https://restapi.amap.com/v3/weather/weatherInfo"
    
    params = {
        "key": gaode_api_key,
        "city": adcode if adcode else location,
        "extensions": "base",  # 可改为 "all" 获取预报
    }
    
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to fetch weather: {response.status_code}"}

def run_weather_query():
    """使用 Qwen3 + 查询天气"""
    messages = [
        {"role": "system", "content": "你是一个智能助手，可以查询天气信息。"},
        {"role": "user", "content": "北京现在天气怎么样？"}
    ]
    
    response = dashscope.Generation.call(
        model="qwen-turbo-latest",  # 可使用 Qwen3 最新版本
        messages=messages,
        tools=[weather_tool],  # 传入工具定义
        tool_choice="auto",  # 让模型决定是否调用工具
    )
    print('response=', response.output.choices[0])
    if response.status_code == HTTPStatus.OK:
        # 检查是否需要调用工具
        if "tool_calls" in response.output.choices[0].message:
            tool_call = response.output.choices[0].message.tool_calls[0]
            print('tool_call=', tool_call)
            if tool_call["function"]["name"] == "get_current_weather":
                # 解析参数并调用高德API
                import json
                args = json.loads(tool_call["function"]["arguments"])
                location = args.get("location", "北京")
                adcode = args.get("adcode", None)
                
                weather_data = get_current_weather(location, adcode)
                print(f"查询结果：{weather_data}")
        else:
            print(response.output.choices[0].message.content)
    else:
        print(f"请求失败: {response.code} - {response.message}")

if __name__ == "__main__":
    run_weather_query()

response= {"finish_reason": "tool_calls", "message": {"role": "assistant", "content": "", "tool_calls": [{"function": {"name": "get_current_weather", "arguments": "{\"location\": \"北京\"}"}, "index": 0, "id": "call_abfa9e03f906412d97592d", "type": "function"}]}}
tool_call= {'function': {'name': 'get_current_weather', 'arguments': '{"location": "北京"}'}, 'index': 0, 'id': 'call_abfa9e03f906412d97592d', 'type': 'function'}
查询结果：{'status': '1', 'count': '1', 'info': 'OK', 'infocode': '10000', 'lives': [{'province': '北京', 'city': '北京市', 'adcode': '110000', 'weather': '晴', 'temperature': '27', 'winddirection': '南', 'windpower': '≤3', 'humidity': '82', 'reporttime': '2025-08-16 20:38:46', 'temperature_float': '27.0', 'humidity_float': '82.0'}]}
